# Import libraries

In [2]:
import pandas as pd
import numpy as np
import re
import operator
import pickle
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize

nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\emirs\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### Load Data

In [3]:
col_list = ["Pageurl", "Title", "Manufacturer", "Model Name"]
dataset = pd.read_csv("marketing_sample_for_ebay_com-ebay_com_product__20210101_20210331__30k_data.csv", usecols=col_list)

### Data Cleaning

In [4]:
dataset['Title']=[str(entry).lower() for entry in dataset['Title']]

In [5]:
dataset['Title'] = dataset['Title'].replace(to_replace='[!"#$%&\'()*+,/:;<=>?@[\\]^_`{|}~]', value=' ', regex=True)
dataset['Title'] = dataset['Title'].replace(to_replace='-', value=' ', regex=True)
dataset['Title'] = dataset['Title'].replace(to_replace='  ', value=' ', regex=True)
dataset['Title'] = dataset['Title'].replace(to_replace='！', value=' ', regex=True)

In [6]:
dataset.head()

,Pageurl,Title,Manufacturer,Model Name
0,https://www.ebay.com/p/7024512090?iid=16461748...,samsung galaxy note 8 6gb ram 64gb dual sim un...,Samsung,Samsung Galaxy Note8
1,https://www.ebay.com/itm/Apple-Airpods-Pro-wit...,apple airpods pro with wireless charging case ...,Apple Airpods Pro with wireless charging case ...,Apple Airpods Pro with wireless charging case ...
2,https://www.ebay.com/p/238211749?iid=174232921...,apple ipad pro 1st gen. 64gb wi fi 4g unlocke...,Apple,Apple iPad Pro (1st Generation)
3,https://www.ebay.com/itm/Nintendo-wii-console-...,nintendo wii console fit board and games bundle,Nintendo wii console fit board and games bundle,Nintendo wii console fit board and games bundle
4,https://www.ebay.com/itm/Fossil-smart-watch/16...,fossil smart watch,Fossil smart watch,Fossil smart watch


In [7]:
dataset['Word tokenize']= [word_tokenize(entry) for entry in dataset['Title']]

## TF-IDF Search Vector

In [8]:
vocabulary = set()

for doc in dataset['Title']:
    vocabulary.update(doc.split(' '))
    
vocabulary = list(vocabulary)

tfidf = TfidfVectorizer(vocabulary=vocabulary, dtype=np.float32)
tfidf.fit(dataset['Title'])
tfidf_tran = tfidf.transform(dataset['Title'])


In [9]:
vocabulary[0:10]

['',
 'satsystem',
 'm3888',
 'psp',
 'shl3060bk',
 'l8star',
 '32ft',
 '8145u',
 'otter',
 'fun']

In [10]:
with open('C:\\Users\\emirs\\PycharmProjects\\tf-idf\\tfidf.pkl', 'wb') as handle:
    pickle.dump(tfidf_tran,handle)

In [11]:
t = pickle.load(open('C:\\Users\\emirs\\PycharmProjects\\tf-idf\\tfidf.pkl', 'rb'))

In [12]:
with open('C:\\Users\\emirs\\PycharmProjects\\tf-idf\\vocabulary_dataset.txt', 'w') as file:
    file.write(str(vocabulary))

UnicodeEncodeError: 'charmap' codec can't encode characters in position 456-459: character maps to <undefined>

In [ ]:
with open("C:\\Users\\emirs\\PycharmProjects\\tf-idf\\vocabulary_dataset.txt", "r") as file:
    data2 = eval(file.readline())

In [15]:
def GenVectorT(tokens):
    Q = np.zeros((len(vocabulary)))
    
    x = tfidf.transform(tokens)
    for token in tokens[0].split(' '):
        try:
            ind = vocabulary.index(token)
            Q[ind] = x[0, tfidf.vocabulary_[token]]
        except:
            pass
    return Q

In [16]:
def CosineSimilarity(a, b):
    cos_sim = np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
    return cos_sim

In [17]:
def CosineSimilarity_T(k, query):
    preprocessed_query = preprocessed_query = re.sub("\W+", " ", query).strip()
    tokens = word_tokenize(str(preprocessed_query))
    
    q_df = pd.DataFrame(columns=['q_clean'])
    q_df.loc[0,'q_clean'] =tokens
    q_df['q_clean'] = dataset['Title']
    
    q_df=q_df.replace(to_replace ="\[.", value = '', regex = True)
    q_df=q_df.replace(to_replace ="'", value = '', regex = True)
    q_df=q_df.replace(to_replace =" ", value = '', regex = True)
    q_df=q_df.replace(to_replace ='\]', value = '', regex = True)
    
    d_cosines = []
    query_vector = GenVectorT(q_df['q_clean'])
    
    for d in tfidf_tran.A:
        d_cosines.append(CosineSimilarity(query_vector, d))
        #print(query_vector)
        #print(d)
        
    out = np.array(d_cosines).argsort()[-k:][::-1]
    d_cosines.sort()
    
    a = pd.DataFrame()
    for i,index in enumerate(out):
        a.loc[i,'index'] = str(index)
        a.loc[i,'Title'] = dataset['Title'][index]
    for j,simScore in enumerate(d_cosines[-k:][::-1]):
        a.loc[j,'Score'] = simScore
    return a

In [18]:
%time CosineSimilarity_T(10,'apple')

C:\Users\emirs\AppData\Local\Temp/ipykernel_20740/3340107106.py:2: RuntimeWarning: invalid value encountered in double_scalars
  cos_sim = np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))


Wall time: 3.41 s


,index,Title,Score
0,29969,apple iphone se 2. gen 64gb schwarz ohne sim...,NaN
1,9993,hampton bay wired door chime in white r29l 6 t...,NaN
2,9981,mecool kd1 android tv stick,NaN
3,9982,beats solo 3 wireless headphones,NaN
4,9983,honeywell 200m wireless doorbell kit with rope...,NaN
5,9984,rlx bluetooth stereo headset blue green,NaN
6,9985,mobile siemens sl 55,NaN
7,9986,qrh00880b security cameras,NaN
8,9987,airbag vest for motorcycle. protection for bac...,NaN
9,9988,apple iphone xr mry62b a a2105 12mp 6.1 64gb d...,NaN


In [19]:
print(CosineSimilarity_T(10,'samsung'))

C:\Users\emirs\AppData\Local\Temp/ipykernel_20740/3340107106.py:2: RuntimeWarning: invalid value encountered in double_scalars
  cos_sim = np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))


   index                                              Title  Score
0  29969  apple iphone se 2. gen  64gb  schwarz ohne sim...    NaN
1   9993  hampton bay wired door chime in white r29l 6 t...    NaN
2   9981                        mecool kd1 android tv stick    NaN
3   9982                   beats solo 3 wireless headphones    NaN
4   9983  honeywell 200m wireless doorbell kit with rope...    NaN
5   9984            rlx bluetooth stereo headset blue green    NaN
6   9985                               mobile siemens sl 55    NaN
7   9986                         qrh00880b security cameras    NaN
8   9987  airbag vest for motorcycle. protection for bac...    NaN
9   9988  apple iphone xr mry62b a a2105 12mp 6.1 64gb d...    NaN
